Scenario: Employee Promotion Decision (Single-Layer Perceptron)
Inputs (2 features):
Years of experience (0 = low, 1 = sufficient)
Performance rating (0 = poor, 1 = good)
Output:
1 = gets promoted
0 = not promoted
Rule (intuitive):
Employee gets promoted only if both experience and performance are good.
This is again an AND logic gate, but in a workplace context.


In [1]:
# creating our own synthetic dataset.
import numpy as np
 
# We are creating a dataset for a promotion decision problem.
# Features (X):
#   X[:,0] -> experience (0 = low, 1 = sufficient)
#   X[:,1] -> performance (0 = poor, 1 = good)
#
# Target (y):
#   1 = employee gets promoted
#   0 = employee does NOT get promoted
#
# Rule used:
#   Promotion happens ONLY when BOTH experience=1 AND performance=1.
#   This is logically the AND gate.
#
# Below we repeat each type 2-3 times to make the dataset slightly larger.
 
# X = input matrix (experience, performance)
X = np.array([
    [0, 0],   # low experience, poor performance → not promoted
    [0, 0],   # repeated sample for training stability
    [0, 1],   # low experience, good performance → still not promoted
    [0, 1],   # repeated
    [1, 0],   # sufficient experience, poor performance → not promoted
    [1, 0],   # repeated
    [1, 1],   # sufficient experience, good performance → promoted
    [1, 1],   # repeated
    [1, 1],   # repeated again (multiple positives)
])
 
# y = output labels (promotion decision)
y = np.array([
    [0],  # employee not promoted
    [0],  # not promoted (repeat)
    [0],  # not promoted (good performance but low experience)
    [0],  # not promoted (repeat)
    [0],  # not promoted (experience OK but performance poor)
    [0],  # not promoted (repeat)
    [1],  # promoted (both conditions satisfied)
    [1],  # promoted (repeat)
    [1],  # promoted (repeat)
])

In [2]:
# 1. Activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)


#initializing hidden weights, inputs and biases.
np.random.seed(1)
weights_input_hidden = np.random.rand(2,2)
weights_hidden_output = np.random.rand(2,1)
bias_hidden = np.zeros((1,2))
bias_output = np.zeros((1,1))

#training settings
epochs = 5000
learning_rate = 0.1

#training of the model
for epoch in range(epochs):
    #forward pass
    # Hidden layer
    hidden_input = np.dot(X, weights_input_hidden) + bias_hidden
    hidden_output = sigmoid(hidden_input)
    
    # Output layer
    final_input = np.dot(hidden_output, weights_hidden_output) + bias_output
    final_output = sigmoid(final_input)
    # ---- Error ----
    error = y - final_output

    # ---- Backpropagation ----
    d_output = error * sigmoid_derivative(final_output)
    d_hidden = d_output.dot(weights_hidden_output.T) * sigmoid_derivative(hidden_output)

    # ---- Update weights and biases ----
    weights_hidden_output += hidden_output.T.dot(d_output) * learning_rate
    bias_output += np.sum(d_output, axis=0, keepdims=True) * learning_rate

    weights_input_hidden += X.T.dot(d_hidden) * learning_rate
    bias_hidden += np.sum(d_hidden, axis=0, keepdims=True) * learning_rate

    # Print loss sometimes
    if epoch % 1000 == 0:
        loss = np.mean(np.square(error))
        print(f"Epoch {epoch}, Loss: {loss:.4f}")


# 5. Predictions after training
print("\nPredictions after training:")
for i, inputs in enumerate(X):
    hidden_output = sigmoid(np.dot(inputs, weights_input_hidden) + bias_hidden)
    final_output = sigmoid(np.dot(hidden_output, weights_hidden_output) + bias_output)
    print(f"{inputs} -> {final_output[0][0]:.3f} (Target: {y[i][0]})")


Epoch 0, Loss: 0.2602
Epoch 1000, Loss: 0.0129
Epoch 2000, Loss: 0.0034
Epoch 3000, Loss: 0.0018
Epoch 4000, Loss: 0.0012

Predictions after training:
[0 0] -> 0.010 (Target: 0)
[0 0] -> 0.010 (Target: 0)
[0 1] -> 0.033 (Target: 0)
[0 1] -> 0.033 (Target: 0)
[1 0] -> 0.034 (Target: 0)
[1 0] -> 0.034 (Target: 0)
[1 1] -> 0.966 (Target: 1)
[1 1] -> 0.966 (Target: 1)
[1 1] -> 0.966 (Target: 1)


    Scenario: Loan Approval Prediction (Multilayer Neural Network)
    Context
    Banks want to decide whether to approve a loan application. The decision depends on multiple factors, and the relationships are non-linear (not just a simple rule).
    Inputs (Features)
    Income level (normalized numeric value)
    Credit score (normalized numeric value)
    Output
    1 = Loan approved
    0 = Loan rejected
    Rule (intuitive, hidden from model)
    Higher income + good credit score → likely approved.
    Low income + poor credit score → likely rejected.
    Middle cases depend on combinations (non-linear patterns).

In [5]:

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report



# ------------------ 2) Synthetic Data ------------------
N = 8000
income = np.clip(np.random.beta(a=2.0, b=2.0, size=N), 0, 1)
credit = np.clip(np.random.beta(a=2.5, b=1.8, size=N), 0, 1)
X = np.c_[income, credit]


def sigmoid(z): return 1 / (1 + np.exp(-z))
z = (
    3.0 * (credit - 0.6) +
    2.0 * (income - 0.5) +
    3.5 * (income * credit - 0.3) +
    1.2 * (credit**2 - 0.35)
)
p_approve = sigmoid(z)
y = (np.random.rand(N) < p_approve).astype(int)


X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)



# ------------------ 3) Build MLP ------------------
model = keras.Sequential([
    layers.Input(shape=(2,)),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=64,
    verbose=0
)

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f'Test accuracy: {acc*100:.2f}%')

Test accuracy: 74.42%


NEW CONTEXT: HOSPITAL PATIENT DETERIORATION PREDICTION
Scenario: ICU Patient Health Risk Forecasting
A hospital ICU continuously monitors critical patient vitals.
The goal is to predict the patient’s risk score for the next 6 hours using the previous 12 hours of multivariate time-series data, so doctors can intervene before a medical emergency occurs.
🔹 Parameters Collected (Every Hour)
Feature	Description
heart_rate	Beats per minute
systolic_bp	Blood pressure (mmHg)
oxygen	Oxygen saturation (%)
resp_rate	Breaths per minute
temperature	Body temperature (°C)
🔹 Sample Dataset (12 Hours)
 
Hour	HR	BP	O₂	Resp	Temp
1	82	120	98	16	36.8
2	85	118	97	17	36.9
3	88	115	96	18	37.0
4	92	112	95	19	37.1
5	96	108	94	20	37.2
6	100	105	93	21	37.4
7	104	102	92	22	37.6
8	108	98	91	23	37.8
9	112	95	90	24	38.0
10	118	92	89	26	38.3
11	122	88	87	28	38.6
12	128	85	85	30	39.0

In [3]:

# =========================
# ICU Risk Forecast (RNN)
# =========================
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import MinMaxScaler



import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


df = pd.read_csv("Datasets/ICU_12h_vitals.csv")  # ensure the file is in your working directory

# 2) Select the 5 vital features (exclude Hour)
features = ["HR", "BP", "O2", "Resp", "Temp"]
X_df = df[features].copy()

# 3) (Optional) Scale features to [0,1] using approximate clinical ranges
#    Adjust ranges if you have site-specific norms.
range_mins = np.array([40, 70, 70, 8, 34.0], dtype=float)   # HR, BP, O2, Resp, Temp
range_maxs = np.array([160, 180, 100, 40, 41.0], dtype=float)

X_norm = (X_df.values - range_mins) / (range_maxs - range_mins)
X_norm = np.clip(X_norm, 0.0, 1.0)

# 4) Reshape to (batch, time_steps, features) = (1,12,5)
X_input = X_norm.reshape(1, 12, 5)

print("Model input shape:", X_input.shape)  # (1, 12, 5)


# --- Build a tiny SimpleRNN model ---
model = models.Sequential([
    layers.SimpleRNN(32, activation='tanh', input_shape=(12, 5)),
    layers.Dense(16, activation='relu'),
    layers.Dense(6, activation='sigmoid')  # 6 risk scores in [0,1]
])

model.compile(optimizer='adam', loss='mse')


y_dummy = np.random.rand(1, 6)  # placeholder; delete when you have labels
model.fit(X_input, y_dummy, epochs=1, verbose=0)

# --- Predict next 6-hour risk ---
pred = model.predict(X_input, verbose=0).flatten()

print("Predicted risk for next 6 hours (0-1):")
for i, r in enumerate(pred, start=1):
    print(f"t+{i}h: {r:.3f}")


Model input shape: (1, 12, 5)


c:\Users\Siddharth.Kshirsagar\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Predicted risk for next 6 hours (0-1):
t+1h: 0.398
t+2h: 0.369
t+3h: 0.555
t+4h: 0.441
t+5h: 0.578
t+6h: 0.540


In [4]:
# Assume X_input is shaped (1, 12, 5) from your CSV preprocessing
# and you have some training windows later; for demo we create dummy targets
X_train = X_input  # (1,12,5) – demo only
y_train = np.random.rand(1, 6).astype("float32")  # demo target for 6 hours

model = models.Sequential([
    layers.SimpleRNN(32, activation='tanh', input_shape=(12, 5)),
    layers.Dense(16, activation='relu'),
    layers.Dense(6, activation='sigmoid')
])

# Compile with regression metrics
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=[tf.keras.metrics.MeanAbsoluteError(name='mae')]
)

# Use validation_split (if you have >1 sample) or provide validation_data
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=1,
    validation_split=0.0,  # set >0 when you have enough samples
    verbose=1
)

# Print final metrics
print("History keys:", history.history.keys())   # contains 'loss' and 'mae'; add val_* when you have validation
for epoch, (loss, mae) in enumerate(zip(history.history['loss'], history.history['mae']), 1):
    print(f"Epoch {epoch}: loss={loss:.4f}, mae={mae:.4f}")


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 962ms/step - loss: 0.0455 - mae: 0.1815
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0412 - mae: 0.1714
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0377 - mae: 0.1626
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0356 - mae: 0.1573
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0336 - mae: 0.1522
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0311 - mae: 0.1454
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0287 - mae: 0.1388
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0265 - mae: 0.1325
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0245 - mae: 0.1272
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0227 - mae: 0.1224
History keys: dict_keys(['loss', 'mae'])
Epoch 1: loss=0.0455, mae=0.1815
Epoch 2: loss=0.0412, mae=0.1714
Epoch 3: loss=0.0377, mae=0.1626
Epoch 4: loss=0.0356, mae=0.1573
Epoch 5: loss=0.0336, mae=0.1522
Epoch 6: los

In [5]:
# Scenario and use case
# Scenario:
# Predict next day’s website traffic (visits) from the last 7 days to help plan server capacity.

# Typical LSTM/GRU uses:

# Time series: stock prices, traffic, energy demand.


# NLP: next-word prediction, translation, sentiment.


# Sequences: clickstreams, sensor readings, speech.


# 1. Data preparation

import numpy as np
import tensorflow as tf

tf.random.set_seed(1)
np.random.seed(1)

# -----------------------------
# 1. Create synthetic traffic data
# -----------------------------
days = np.arange(200)
# baseline 1000 visits, weekly seasonality + small upward trend
traffic = 1000 + 30 * np.sin(2 * np.pi * days / 7) + 0.8 * days

window_size = 7   # use last 7 days -> predict next day

X_list, y_list = [], []
for i in range(len(traffic) - window_size):
    X_list.append(traffic[i:i + window_size])
    y_list.append(traffic[i + window_size])

X = np.array(X_list)            # (samples, 7)
y = np.array(y_list)            # (samples,)

# Normalize (simple)
mean = X.mean()
std = X.std()
X_norm = (X - mean) / std
y_norm = (y - mean) / std

# Reshape for RNN: (batch, time_steps, features)
X_norm = X_norm[..., np.newaxis]   # (samples, 7, 1)
y_norm = y_norm[..., np.newaxis]   # (samples, 1)

# Train/validation split
split = int(0.8 * len(X_norm))
X_train, X_val = X_norm[:split], X_norm[split:]
y_train, y_val = y_norm[:split], y_norm[split:]

# This creates a realistic-looking daily traffic series with weekly seasonality and a trend.

# 2. LSTM architecture and working model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# -----------------------------
# 2. Build LSTM model
# -----------------------------
def build_lstm_model():
    model = Sequential([
        LSTM(
            units=32,
            activation='tanh',
            recurrent_activation='sigmoid',
            input_shape=(window_size, 1)
        ),
        Dense(1)  # predict normalized next-day traffic
    ])
    return model

lstm_model = build_lstm_model()
lstm_model.summary()

# LSTM cell behaviour (intuitive):

# Maintains a cell state that flows along time steps, plus a hidden state.


# Uses input, forget, and output gates to decide

# what new information to add,

# what to erase,

# what to output at each time step.


# This design helps keep important gradients from vanishing across many steps.

# 3. Train LSTM (optimizer + learning rate)
# -----------------------------
# 3. Compile with Adam optimizer
# -----------------------------
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)  # adaptive GD [web:51]

lstm_model.compile(
    optimizer=optimizer,
    loss='mse',
    metrics=['mae']
)

# -----------------------------
# 4. Train the model
# -----------------------------
history_lstm = lstm_model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=16,
    validation_data=(X_val, y_val),
    verbose=0
)

print("LSTM final train loss:", history_lstm.history['loss'][-1])
print("LSTM final val loss  :", history_lstm.history['val_loss'][-1])
# Adam performs gradient descent with adaptive learning rates and momentum, well suited for LSTM training.


# Learning rate 0.005 balances stability and speed; adjusting it changes convergence behaviour.

# 4. GRU model on the same data

from tensorflow.keras.layers import GRU

# -----------------------------
# 5. Build GRU model
# -----------------------------
def build_gru_model():
    model = Sequential([
        GRU(
            units=32,
            activation='tanh',
            recurrent_activation='sigmoid',
            input_shape=(window_size, 1)
        ),
        Dense(1)
    ])
    return model

gru_model = build_gru_model()

gru_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
    loss='mse',
    metrics=['mae']
)

history_gru = gru_model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=16,
    validation_data=(X_val, y_val),
    verbose=0
)

print("GRU final train loss:", history_gru.history['loss'][-1])
print("GRU final val loss  :", history_gru.history['val_loss'][-1])

# GRU uses update and reset gates with a single hidden state, merging some LSTM gates.


# For the same units, GRU has fewer parameters and is usually faster, often with similar accuracy.


# 5. Prediction and LSTM vs GRU comparison

# -----------------------------
# 6. Predict next day's traffic
# -----------------------------
last_week = traffic[-window_size:]
last_week_norm = ((last_week - mean) / std).reshape(1, window_size, 1)

pred_lstm_norm = lstm_model.predict(last_week_norm, verbose=0)[0, 0]
pred_gru_norm = gru_model.predict(last_week_norm, verbose=0)[0, 0]

pred_lstm = pred_lstm_norm * std + mean
pred_gru = pred_gru_norm * std + mean

print("Last 7 days traffic:", np.round(last_week, 1))
print(f"LSTM predicted next day traffic: {pred_lstm:.1f} visits")
print(f"GRU  predicted next day traffic: {pred_gru:.1f} visits")

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 32)             │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,385 (17.13 KB)

 Trainable params: 4,385 (17.13 KB)

 Non-trainable params: 0 (0.00 B)

LSTM final train loss: 0.0004062442167196423
LSTM final val loss  : 0.010700535960495472
GRU final train loss: 0.0001427667011739686
GRU final val loss  : 0.010459857992827892
Last 7 days traffic: [1141.4 1126.  1132.5 1156.8 1181.1 1187.6 1172.2]
LSTM predicted next day traffic: 1145.9 visits
GRU  predicted next day traffic: 1142.7 visits


📌 Business Scenario
A smart city power department wants to predict tomorrow’s electricity demand (in MW) using the last 7 days of hourly consumption data.
This helps to:
Avoid power shortages ⚡
Optimize power generation scheduling
Reduce operational costs
🔹 Sample Dataset (Simplified – Daily Average)
 
Day	Electricity Demand (MW)
1	320
2	340
3	360
4	355
5	370
6	390
7	410
8	Target (to predict)

In [ ]:

# ===========================
# Smart City Demand Forecast
# LSTM & GRU
# ===========================
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import MinMaxScaler
import math

# --------------------------
# 1) Load the 100-day dataset
# --------------------------
df = pd.read_csv("Datasets/smartcity_power_100days.csv")
series = df["Electricity_Demand_MW"].astype(float).values  # shape: (100,)

# --------------------------
# 2) Train/Test split (chronological)
#    Use first 85 days to train/validate, last 15 days to test
# --------------------------
N = len(series)                 # 100
train_size = 85
train_series = series[:train_size].reshape(-1, 1)
test_series  = series[train_size:].reshape(-1, 1)

# --------------------------
# 3) Scale to [0,1] using train-only fit (avoid leakage)
# --------------------------
scaler = MinMaxScaler()
scaler.fit(train_series)
series_scaled = scaler.transform(series.reshape(-1, 1)).flatten()  # scaled entire series (for windowing)

# --------------------------
# 4) Make sliding windows: last 7 days -> next day
# --------------------------
WINDOW = 100

def make_windows(arr, win=100):
    X, y = [], []
    # arr is 1D scaled series
    for t in range(win, len(arr)):
        hist = arr[t-win:t]     # last 7 values
        target = arr[t]         # next value
        X.append(hist)
        y.append(target)
    return np.array(X), np.array(y)

X_all, y_all = make_windows(series_scaled, WINDOW)  # X_all: (N-7, 7), y_all: (N-7,)

# Indices of each sample correspond to targets at positions [7..N-1]
target_indices = np.arange(WINDOW, N)

# Train samples are those whose target day < train_size
train_mask = target_indices < train_size
test_mask  = target_indices >= train_size

X_train, y_train = X_all[train_mask], y_all[train_mask]
X_test,  y_test  = X_all[test_mask],  y_all[test_mask]

# Reshape for RNN: (samples, time_steps, features)
X_train = X_train.reshape(-1, WINDOW, 1)
X_test  = X_test.reshape(-1, WINDOW, 1)

print("Train samples:", X_train.shape, " Test samples:", X_test.shape)

# --------------------------
# 5) Build two tiny models: LSTM and GRU
# --------------------------
def build_lstm():
    model = models.Sequential([
        layers.LSTM(32, input_shape=(WINDOW, 1)),
        layers.Dense(16, activation="relu"),
        layers.Dense(1)  # regression
    ])
    model.compile(optimizer="adam", loss="mse",
                  metrics=[tf.keras.metrics.MeanAbsoluteError(name="mae")])
    return model

def build_gru():
    model = models.Sequential([
        layers.GRU(32, input_shape=(WINDOW, 1)),
        layers.Dense(16, activation="relu"),
        layers.Dense(1)
    ])
    model.compile(optimizer="adam", loss="mse",
                  metrics=[tf.keras.metrics.MeanAbsoluteError(name="mae")])
    return model

lstm = build_lstm()
gru  = build_gru()

hist_lstm = lstm.fit(
    X_train, y_train,
    epochs=100,
    batch_size=8,
    validation_split=0.2,
    verbose=1,
)

hist_gru = gru.fit(
    X_train, y_train,
    epochs=100,
    batch_size=8,
    validation_split=0.2,
    verbose=1,
)

# --------------------------
# 7) Evaluate on TEST (metrics on original MW scale)
# --------------------------
def inverse_scale(arr_1d):
    # arr_1d is shape (n,) or (n,1) in scaled space -> back to MW
    return scaler.inverse_transform(np.array(arr_1d).reshape(-1,1)).flatten()

# Predictions on test set
yhat_lstm_test = lstm.predict(X_test, verbose=0).flatten()
yhat_gru_test  = gru.predict(X_test,  verbose=0).flatten()

# Inverse transform to MW
y_test_MW       = inverse_scale(y_test)
yhat_lstm_MW    = inverse_scale(yhat_lstm_test)
yhat_gru_MW     = inverse_scale(yhat_gru_test)

# Metrics
def mae(a, b):   return np.mean(np.abs(a - b))
def rmse(a, b):  return math.sqrt(np.mean((a - b)**2))
def mape(a, b):  return np.mean(np.abs((a - b) / (np.clip(a, 1e-8, None)))) * 100

print("\n=== TEST METRICS (Original MW scale) ===")
print(f"LSTM -> MAE: {mae(y_test_MW, yhat_lstm_MW):.2f} | RMSE: {rmse(y_test_MW, yhat_lstm_MW):.2f} | MAPE: {mape(y_test_MW, yhat_lstm_MW):.2f}%")
print(f"GRU  -> MAE: {mae(y_test_MW, yhat_gru_MW):.2f}  | RMSE: {rmse(y_test_MW, yhat_gru_MW):.2f}  | MAPE: {mape(y_test_MW, yhat_gru_MW):.2f}%")

# --------------------------
# 8) Print training history (loss & val_loss)
# --------------------------
def print_history(name, h):
    last_epoch = len(h.history["loss"])
    print(f"\n[{name}] epochs trained: {last_epoch}")
    for k in h.history.keys():
        if k.startswith("val_"):
            continue
    # print last values
    print(f"[{name}] final loss: {h.history['loss'][-1]:.6f} | final val_loss: {h.history['val_loss'][-1]:.6f}")
    if "mae" in h.history:
        print(f"[{name}] final mae:  {h.history['mae'][-1]:.6f}  | final val_mae: {h.history['val_mae'][-1]:.6f}")

print_history("LSTM", hist_lstm)
print_history("GRU",  hist_gru)

# --------------------------
# 9) Predict Day 101
#    Use the last 7 days (Days 94..100) as input
# --------------------------
last7_scaled = series_scaled[-WINDOW:].reshape(1, WINDOW, 1)

pred101_lstm_scaled = lstm.predict(last7_scaled, verbose=0)
pred101_gru_scaled  = gru.predict(last7_scaled,  verbose=0)

pred101_lstm = inverse_scale(pred101_lstm_scaled)[0]
pred101_gru  = inverse_scale(pred101_gru_scaled)[0]
pred101_avg  = (pred101_lstm + pred101_gru) / 2.0

print("\n=== FORECAST: Day 101 ===")
print(f"LSTM prediction (MW): {pred101_lstm:.2f}")
print(f"GRU  prediction (MW): {pred101_gru:.2f}")
print(f"Ensemble average     : {pred101_avg:.2f}")

# (Optional) Save forecasts
out = pd.DataFrame({
    "model": ["LSTM", "GRU", "Avg(LSTM,GRU)"],
    "day": [101, 101, 101],
    "prediction_MW": [pred101_lstm, pred101_gru, pred101_avg]
})
out.to_csv("day101_forecasts.csv", index=False)
print("\nSaved: day101_forecasts.csv")


Train samples: (0, 100, 1)  Test samples: (0, 100, 1)


c:\Users\Siddharth.Kshirsagar\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Training data contains 0 samples, which is not sufficient to split it into a validation and training set as specified by `validation_split=0.2`. Either provide more data, or a different value for the `validation_split` argument.

In [9]:
import os
# 1. Set the download directory to D drive BEFORE importing gensim.downloader
os.environ['GENSIM_DATA_DIR'] = r'D:\gensim-data'
 
import gensim.downloader as api
 
# 2. This will now download the 1.6GB model to D:\gensim-data
model = api.load("word2vec-google-news-300")
 
# Rest of your code
vector = model['python']
similar = model.most_similar('python', topn=5)
print(similar)
 

An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


c:\Users\Siddharth.Kshirsagar\AppData\Local\Programs\Python\Python39\lib\site-packages\google\api_core\_python_version_support.py:237: FutureWarning: You are using a non-supported Python version (3.9.7). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
c:\Users\Siddharth.Kshirsagar\AppData\Local\Programs\Python\Python39\lib\site-packages\google\auth\__init__.py:54: FutureWarning: 
    You are using a Python version 3.9 past its end of life. Google will update
    google-auth with critical bug fixes on a best-effort basis, but not
    with any other fixes or features. Please upgrade your Python version,
    and then update google-auth.
    
  warnings.warn(eol_message.format("3.9"), FutureWarning)
c:\Users\Siddharth.Kshirsagar\AppData\Local\Programs\Python\Python39\lib\site-packages\google\oauth2\__init_

[==================================================] 100.0% 1662.8/1662.8MB downloaded
[('pythons', 0.66883784532547), ('Burmese_python', 0.6680365800857544), ('snake', 0.6606292724609375), ('crocodile', 0.6591362953186035), ('boa_constrictor', 0.6443520188331604)]


In [13]:
similar = model.most_similar(positive=['Russia', 'India'],
    negative=['Trump'], topn=5)
# Returns: [('Washington', 0.68)]
print(similar)
 

[('China', 0.5742051601409912), ('Pakistan', 0.5350226163864136), ('Bangladesh', 0.5193105340003967), ('erstwhile_USSR', 0.5165647268295288), ('Indiaâ_€_™', 0.5160695314407349)]


In [ ]:
from gensim.models import Word2Vec

sentences = [['Machine','learning','is','fun'],['deep','learning','is','bad']]

model = Word2Vec(sentences,sg=1,vector_size=100, window=5,min_count=1, workers=4)

In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
 
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, LSTM
from tensorflow.keras.callbacks import EarlyStopping
 
df = pd.read_csv("Datasets/electricity_daily_avg.csv").sort_values("Day").reset_index(drop=True)
target_col = "Electricity Demand (MW)"  
 
series = df[target_col].to_numpy(dtype=np.float32).reshape(-1, 1)
 
N = len(series)
train_size = int(N * 0.70)
val_size = int(N * 0.15)
 
train = series[:train_size]
val   = series[train_size:train_size + val_size]
test  = series[train_size + val_size:]
 
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
val_scaled   = scaler.transform(val)
test_scaled  = scaler.transform(test)
 
def make_sequences(data, look_back):
    data = np.asarray(data)
    if data.ndim == 1:
        data = data.reshape(-1, 1)
 
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, :])  
        y.append(data[i+look_back, 0])    
 
    X = np.array(X, dtype=np.float32)      
    y = np.array(y, dtype=np.float32)        
    return X, y
 
LOOK_BACK = 7
X_train, y_train = make_sequences(train_scaled, LOOK_BACK)
X_val, y_val     = make_sequences(val_scaled, LOOK_BACK)
X_test, y_test   = make_sequences(test_scaled, LOOK_BACK)
 
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_val  :", X_val.shape,   "y_val  :", y_val.shape)
print("X_test :", X_test.shape,  "y_test :", y_test.shape)
 
tf.keras.utils.set_random_seed(42)
 
 
model = Sequential([
    GRU(64, return_sequences=True, input_shape=(LOOK_BACK, 1)),
    Dropout(0.2),
 
    LSTM(32, return_sequences=False),   # <- LSTM added here
    Dropout(0.2),
 
    Dense(1)
])
 
model.compile(optimizer="adam", loss="mse")
 
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=150,
    batch_size=16,
    callbacks=[EarlyStopping(patience=15, restore_best_weights=True)],
    verbose=1
)
 
 
full_scaled = scaler.transform(series)  
last_seq = full_scaled[-LOOK_BACK:].reshape(1, LOOK_BACK, 1)
tomorrow_scaled = model.predict(last_seq)
tomorrow_mw = scaler.inverse_transform(tomorrow_scaled)[0, 0]
 
print(f"Predicted demand for tomorrow: {tomorrow_mw:.2f} MW")
 

X_train: (77, 7, 1) y_train: (77,)
X_val  : (11, 7, 1) y_val  : (11,)
X_test : (11, 7, 1) y_test : (11,)


c:\Users\Siddharth.Kshirsagar\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 183ms/step - loss: 0.3941 - val_loss: 0.2942
Epoch 2/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.1651 - val_loss: 0.1146
Epoch 3/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0806 - val_loss: 0.1418
Epoch 4/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0991 - val_loss: 0.1149
Epoch 5/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0759 - val_loss: 0.1168
Epoch 6/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0782 - val_loss: 0.1247
Epoch 7/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0729 - val_loss: 0.1194
Epoch 8/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0913 - val_loss: 0.1127
Epoch 9/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0821 - val_loss: 0.1099
Epoch 10/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0823 - val_loss: 0.1097
Epoch 11/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0687 - val_loss: 0.1102
Epoch 12/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0780 - val_l

📌 Business Scenario
A smart city power department wants to predict tomorrow’s electricity demand (in MW) using the last 7 days of hourly consumption data.
This helps to:
Avoid power shortages ⚡
Optimize power generation scheduling
Reduce operational costs
🔹 Sample Dataset (Simplified – Daily Average)
 
Day	Electricity Demand (MW)
1	320
2	340
3	360
4	355
5	370
6	390
7	410
8	Target (to predict)
 
 
Use LSTM and GRU model for forcasting
 

In [ ]:

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense


tf.random.set_seed(42)
np.random.seed(42)


series = np.array([320, 340, 360, 345, 370, 390, 410], dtype=np.float32)

window_size = 7  
assert len(series) == window_size, "We only have the 7 days that form a single training window."


X = series.reshape(1, window_size, 1)  # shape: (batch=1, time=7, features=1)

last_diff = float(series[-1] - series[-2])  # 410 - 390 = 20
y_value = float(series[-1] + last_diff)     
y = np.array([[y_value]], dtype=np.float32)  


x_mean = X.mean()
x_std  = X.std() + 1e-8
Xn = (X - x_mean) / x_std


def build_lstm():
    model = Sequential([
        LSTM(32, input_shape=(window_size, 1)),
        Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2, clipnorm=1.0),
                  loss='mse')
    return model

def build_gru():
    model = Sequential([
        GRU(32, input_shape=(window_size, 1)),
        Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2, clipnorm=1.0),
                  loss='mse')
    return model

lstm_model = build_lstm()
gru_model  = build_gru()

lstm_model.summary()
gru_model.summary()


epochs = 300
history_lstm = lstm_model.fit(Xn, y, epochs=epochs, batch_size=1, verbose=0)
history_gru  = gru_model.fit( Xn, y, epochs=epochs, batch_size=1, verbose=0)

print("LSTM final train loss:", float(history_lstm.history['loss'][-1]))
print("GRU  final train loss:", float(history_gru.history['loss'][-1]))


pred_lstm = float(lstm_model.predict(Xn, verbose=0)[0, 0])
pred_gru  = float(gru_model.predict( Xn, verbose=0)[0, 0])

naive_repeat_last = float(series[-1])          
naive_add_last_diff = float(series[-1] + last_diff)  


c:\Users\Siddharth.Kshirsagar\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                   │ (None, 32)             │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,385 (17.13 KB)

 Trainable params: 4,385 (17.13 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_5 (GRU)                     │ (None, 32)             │         3,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,393 (13.25 KB)

 Trainable params: 3,393 (13.25 KB)

 Non-trainable params: 0 (0.00 B)

LSTM final train loss: 105510.5390625
GRU  final train loss: 107403.5
